<a href="https://colab.research.google.com/github/aditivin16/Fake-News-Classifier/blob/main/fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

df=pd.read_csv('/content/drive/My Drive/Fake_News/train.csv')
df.head(10)


,id,title,text,label,all
0,18,MAJOR LIBERAL RAG RELUCTANTLY PUBLISHES Articl...,"The Atlantic, a publication that wouldn t know...",1,MAJOR LIBERAL RAG RELUCTANTLY PUBLISHES Articl...
1,48,"AT&T, Time Warner and the Death of Privacy","AT&T, Time Warner and the Death of Privacy \...",1,"AT&T, Time Warner and the Death of PrivacyAT&T..."
2,104,The World Sees a Diminished America,The World Sees a Diminished America November 1...,1,The World Sees a Diminished AmericaThe World S...
3,145,The Bolton Threat to Trump’s Middle East Policy,Written by Gareth Porter Post-election comme...,1,The Bolton Threat to Trump’s Middle East Polic...
4,146,Life at migrant centers in Germany: RT talks t...,Life at migrant centers in Germany: RT talks t...,1,Life at migrant centers in Germany: RT talks t...
5,256,Liberalism Past: Not Being an Idiot,by Outis Philalithopoulos By Outis Philalithop...,1,Liberalism Past: Not Being an Idiotby Outis Ph...
6,286,THE LIST OF 65 MAINSTREAM MEDIA “Journalists” ...,Many of the hacks on this list like Rachel Mad...,1,THE LIST OF 65 MAINSTREAM MEDIA “Journalists” ...
7,326,ILLEGAL INVASION CONTINUES: NYC RAMPING UP TO ...,Insane! This really is the definition of insan...,1,ILLEGAL INVASION CONTINUES: NYC RAMPING UP TO ...
8,400,IS TIM ALLEN’S “Last Man Standing” About To Be...,Tim Allen took to Twitter to express his disap...,1,IS TIM ALLEN’S “Last Man Standing” About To Be...
9,561,World's Oldest Person Had Smoked For 76 Years,Report Copyright Violation World's Oldest Pers...,1,World's Oldest Person Had Smoked For 76 YearsR...


In [3]:
df=df.dropna()

In [4]:
X=df.drop('label',axis=1)
y=df['label']

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [6]:
### Vocabulary size
voc_size=5000

In [7]:
messages=X.copy()
messages['title'][1]
messages.reset_index(inplace=True)


In [8]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [9]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
### Dataset Preprocessing
def preprocess_text(text):
    lem = WordNetLemmatizer()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = nltk.word_tokenize(text)
    words = [lem.lemmatize(word) for word in words if word not in stopwords.words('english')]
    preprocessed_text = ' '.join(words)
    return preprocessed_text

# Assuming `messages` is your DataFrame containing text data
corpus = messages['title'].apply(preprocess_text).tolist()


In [11]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr



[[3552,
  91,
  1268,
  1024,
  207,
  2625,
  491,
  1959,
  2983,
  574,
  4060,
  2437,
  3085,
  3128],
 [2321, 624, 1334, 2109],
 [3614, 897, 2519, 4937],
 [4845, 4292, 1959, 3512, 2367, 1042],
 [3887, 3595, 4436, 4148, 4397, 4296, 3183, 4898, 2738, 3709],
 [1887, 4053, 4387],
 [3798,
  3513,
  2704,
  926,
  3343,
  3231,
  1022,
  1352,
  2704,
  3568,
  4816,
  3609,
  1407,
  1241],
 [796, 4262, 1533, 1227, 2066, 4136, 3413, 1990, 3260, 4402],
 [2408,
  556,
  442,
  286,
  3108,
  4593,
  1416,
  1932,
  3966,
  966,
  4599,
  716,
  3299,
  2893],
 [3614, 1448, 4903, 3547, 943],
 [1959, 2123, 3569, 3935, 2117, 1042, 855],
 [2041, 924, 3614, 2137, 3180, 2203, 2892, 3180, 1961, 1155, 183, 869],
 [2633, 4377, 4407, 2565, 3299, 102, 3042, 1384, 2893],
 [3535, 1915, 37, 2526, 3413, 1792, 1253, 2246, 2130, 2469],
 [924, 398, 535, 3979, 2597, 1693, 495, 4415, 535, 3945, 1959, 1573, 910, 869],
 [1959, 1606, 2273, 3642, 3227, 4500],
 [4765, 333, 4226, 4926, 748, 1959, 3965, 1686, 355

In [12]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2437 3085 3128]
 [   0    0    0 ...  624 1334 2109]
 [   0    0    0 ...  897 2519 4937]
 ...
 [   0    0    0 ...  164 4113  216]
 [   0    0    0 ...  410 1384 4174]
 [   0    0    0 ... 4045 4413 3935]]


In [13]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0, 3552,   91, 1268, 1024,  207,
       2625,  491, 1959, 2983,  574, 4060, 2437, 3085, 3128], dtype=int32)

In [14]:
len(embedded_docs),y.shape

(9550, (9550,))

In [15]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [16]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
X_final.shape,y_final.shape

((9550, 20), (9550,))

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [19]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
120/120 [==============================] - 7s 39ms/step - loss: 0.4009 - accuracy: 0.8626 - val_loss: 0.3132 - val_accuracy: 0.8775
Epoch 2/10
120/120 [==============================] - 7s 55ms/step - loss: 0.2404 - accuracy: 0.9026 - val_loss: 0.2604 - val_accuracy: 0.9047
Epoch 3/10
120/120 [==============================] - 4s 35ms/step - loss: 0.1641 - accuracy: 0.9326 - val_loss: 0.2885 - val_accuracy: 0.9016
Epoch 4/10
120/120 [==============================] - 4s 34ms/step - loss: 0.1155 - accuracy: 0.9552 - val_loss: 0.3537 - val_accuracy: 0.8995
Epoch 5/10
120/120 [==============================] - 6s 50ms/step - loss: 0.0857 - accuracy: 0.9671 - val_loss: 0.4136 - val_accuracy: 0.8827
Epoch 6/10
120/120 [==============================] - 5s 41ms/step - loss: 0.0649 - accuracy: 0.9776 - val_loss: 0.4163 - val_accuracy: 0.8885
Epoch 7/10
120/120 [==============================] - 4s 34ms/step - loss: 0.0491 - accuracy: 0.9844 - val_loss: 0.4030 - val_accuracy: 0.8874

In [20]:
y_pred_1=model.predict(X_test)
threshold = 0.5
y_pred=(y_pred_1>threshold).astype(int)
print(np.shape(y_pred))

60/60 [==============================] - 1s 7ms/step
(1910, 1)


In [21]:
count=0
for i in range(len(y_pred)):
  if(y_pred[i]==0):
    count+=1
print(count)

261


In [22]:
from sklearn.metrics import confusion_matrix

In [23]:
confusion_matrix(y_test,y_pred)

array([[ 138,  119],
       [ 123, 1530]])

In [24]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7312774086026821

In [26]:
df_test=pd.read_csv('/content/drive/My Drive/Fake_News/test.csv')
df_test.head()
df_test=df_test.dropna()
messages_test=df_test.copy()
messages_test['title'][1]
messages_test.reset_index(inplace=True)
### Dataset Preprocessing
def preprocess_text(text):
    lem = WordNetLemmatizer()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = nltk.word_tokenize(text)
    words = [lem.lemmatize(word) for word in words if word not in stopwords.words('english')]
    preprocessed_text = ' '.join(words)
    return preprocessed_text

# Assuming `messages_test` is your DataFrame containing text data
corpus_test = messages_test['title'].apply(preprocess_text).tolist()

# Vocabulary size for one-hot encoding
voc_size = 5000  # Adjust this based on your vocabulary size

onehot_repr_test = [one_hot(words, voc_size) for words in corpus_test]

# Adjust `sent_length` based on your desired sentence length
sent_length = 20

embedded_docs_test = pad_sequences(onehot_repr_test, padding='pre', maxlen=sent_length)

print(embedded_docs_test)
x_test = np.array(embedded_docs_test)


[[   0    0    0 ...   16 1683  943]
 [   0    0    0 ... 1040 3036 3887]
 [   0    0    0 ... 4299  604 4602]
 ...
 [   0    0    0 ... 3965  455 2321]
 [   0    0    0 ... 3965  455 2321]
 [   0    0    0 ... 1756 4897   27]]


In [ ]:
y_test_pred_1=model.predict(x_test)
threshold = 0.5
y_test_pred=(y_test_pred_1>threshold).astype(int)

print(np.shape(y_test_pred))
print((y_test_pred))

In [ ]:
count=0
for i in range(len(y_test_pred)):
  if(y_test_pred[i]==0):
    count+=1
print(count)

In [ ]:
data = {'id':df_test["id"],'label':y_test_pred[:,0]}
print(len(df_test["id"]))
print(len(y_test_pred[:,0]))
sub_df = pd.DataFrame(data)
sub_df.tail()

In [ ]:
sub_df.to_csv("/content/drive/MyDrive/Fake_News/submission3.csv",index=False)